# Data Loading And Analysis

In [ ]:
import os
import cv2
import json
from tqdm import tqdm
from pathlib import Path
import numpy as np
from typing import List,Tuple,Dict

In [7]:
def get_mask(size, label: dict) -> np.ndarray:
    mask = np.zeros(size)
    poly = np.array(label["quad"])
    return cv2.fillPoly(mask, [poly], (255,))

def main():
    # Ensure output directories exist or create them
    output_path = '.'  # Replace with args.output_path if using argparse
    image_output_path = os.path.join(output_path, "images")
    mask_output_path = os.path.join(output_path, "masks")
    os.makedirs(image_output_path, exist_ok=True)
    os.makedirs(mask_output_path, exist_ok=True)

    input_path = '../../midv500_data/midv500'
    images = sorted(Path(input_path).rglob("*.tif"))
    labels = sorted(Path(input_path).rglob("*.json"))

    images_id2path = paths2ids(images)
    labels_id2path = paths2ids(labels)

    for image_id in tqdm(images_id2path.keys()):
        if image_id not in labels_id2path:
            continue
        image_path = images_id2path[image_id]
        label_path = labels_id2path[image_id]

        with open(label_path) as f:
            label = json.load(f)

        if "quad" not in label:
            continue

        image = cv2.imread(str(image_path))

        height, width = image.shape[:2]

        # Assuming get_mask() is defined elsewhere to generate masks
        mask = get_mask(image,(height, width), label)
        if  mask is None:
            print("skipping the frame",images)
            continue

        # Save image and mask
        cv2.imwrite(os.path.join(image_output_path, f"{image_id}.jpg"), image)
        cv2.imwrite(os.path.join(mask_output_path, f"{image_id}.png"), mask)

def paths2ids(paths):
    # Example function to create a dictionary of paths to IDs
    id2path = {}
    for path in paths:
        # Assuming IDs are derived from file names or paths
        file_name = Path(path).stem  # Get the file name without extension
        id2path[file_name] = path
    return id2path

# if __name__ == "__main__":
#     main()


In [4]:

def paths2ids(paths: List[Path]) -> Dict[str, Path]:
    return {x.stem: x for x in paths}

def get_mask(image,size: Tuple[int, int], label: Dict[str, list]) -> np.ndarray:
    # mask = np.zeros(size, dtype=np.uint8)
    # print(label)
    if "quad" in label:
        cropped_image = crop_quad(image,label["quad"])
        # cv2.rectange(image, (x_min, y_min), (x_max, y_max), (255,), 4)
    
    return cropped_image


#checking bounding box by cropping
def crop_quad(image, quad_coords):
    # Convert quad_coords to NumPy array
    quad_coords = np.array(quad_coords, dtype=np.int32)
    if np.any(quad_coords < 0):
        print("Ignoring image: Negative coordinates found in quadrilateral.")
        return None
    
    # Compute bounding box for the quad
    x, y, w, h = cv2.boundingRect(quad_coords)
    
    # Create a mask for the quad
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, [quad_coords], (255, 255, 255))
    
    # Bitwise AND between mask and image
    masked_image = cv2.bitwise_and(image, mask)
    
    # Crop the masked image using the bounding box
    cropped_image = masked_image[y:y+h, x:x+w]
    
    return cropped_image

## Split creation

In [2]:
import os
import cv2
import json
from tqdm import tqdm
from pathlib import Path

In [5]:

# Ensure output directories exist or create them
output_path = '.'  # Replace with args.output_path if using argparse
image_output_path = os.path.join(output_path, "images")
mask_output_path = os.path.join(output_path, "masks")
os.makedirs(image_output_path, exist_ok=True)
os.makedirs(mask_output_path, exist_ok=True)

input_path = '../../midv500_data/midv500'
images = sorted(Path(input_path).rglob("*.tif"))
labels = sorted(Path(input_path).rglob("*.json"))

images_id2path = paths2ids(images)
labels_id2path = paths2ids(labels)

In [6]:

dataset_1 = {"id":[],"passport":[],"drvlic":[],"other_ids":[]}

for image_id in tqdm(images_id2path.keys()):
    if image_id not in labels_id2path:
        continue

    image_path = images_id2path[image_id]
    label_path = labels_id2path[image_id]

    with open(label_path) as f:
        label = json.load(f)

    if "quad" not in label:
        continue
    
    
    name = str(image_path).split('/')[-4]
    image_path,label_path = str(image_path),str(label_path)
    if 'id' in name:
        dataset_1['id'].append([image_path,label_path])
    elif 'passport' in name:
        dataset_1['passport'].append([image_path,label_path])
    elif 'drvlic' in name:
        dataset_1['drvlic'].append([image_path,label_path])
    else:
        dataset_1['other_ids'].append([image_path,label_path])


100%|██████████| 15050/15050 [00:01<00:00, 7537.89it/s]


In [7]:
#creating train,val and test file for all the model versions

#taking train==70%, val=15%,test=15%
train_p = 0.7
val_p = 0.15
test_p = 0.15

dataset_train = {"id":[],"passport":[],"drvlic":[],"other_ids":[]}

dataset_val = {"id":[],"passport":[],"drvlic":[],"other_ids":[]}

dataset_test = {"id":[],"passport":[],"drvlic":[],"other_ids":[]}

for card_name,val in dataset_1.items():
    lenth = len(dataset_1[card_name])
    values = dataset_1[card_name]
    
    train_length = int(lenth*train_p)
    val_length = int(lenth*val_p)
    test_length = int(lenth*test_p)

    dataset_train[card_name] = values[0:train_length]
    dataset_val[card_name] = values[train_length:val_length+train_length]
    dataset_test[card_name] = values[train_length+val_length:]


In [63]:
# Save dataset_1 as JSON'
final_dataset = {"train":dataset_train,"val":dataset_val,"test":dataset_test}
json_filename = 'dataset_all.json'
with open(json_filename, 'w') as json_file:
    json.dump(final_dataset, json_file, indent=4)

print(f"Dataset saved to {json_filename}")

Dataset saved to dataset_all.json


In [8]:
for card_name,val in dataset_1.items():
    print(card_name)
    print(len(val))

id
5700
passport
4800
drvlic
3600
other_ids
900


In [57]:
for card_name,val in dataset_train.items():
    print(card_name)
    print(len(val))

id
3989
passport
3360
drvlic
2520
other_ids
630


In [58]:
for card_name,val in dataset_val.items():
    print(card_name)
    print(len(val))

id
855
passport
720
drvlic
540
other_ids
135


In [59]:
for card_name,val in dataset_test.items():
    print(card_name)
    print(len(val))

id
856
passport
720
drvlic
540
other_ids
135


In [77]:
import os
import cv2
import json
from tqdm import tqdm
from pathlib import Path

def main(data,output_path):
    # Ensure output directories exist or create them
    #output_path = '../../train'  # Replace with args.output_path if using argparse
    image_output_path = os.path.join(output_path, "images")
    mask_output_path = os.path.join(output_path, "masks")
    os.makedirs(image_output_path, exist_ok=True)
    os.makedirs(mask_output_path, exist_ok=True)
    
    for key in data.keys():
        for i  in tqdm(range(len(data[key]))):
            # print(data[key][i])
            image_path = data[key][i][0]#images_id2path[image_id]
            label_path = data[key][i][1]#labels_id2path[image_id]

            with open(label_path) as f:
                label = json.load(f)
                
            if "quad" not in label:
                continue
            
            bbox =  np.array(label["quad"])
            if len(bbox[bbox< 0]): #removing all negative bboxes.
                continue
            image = cv2.imread(str(image_path))

            height, width = image.shape[:2]

            # Assuming get_mask() is defined elsewhere to generate masks
            mask = get_mask((height, width), label)
            if  mask is None:
                print("skipping the frame",images)
                continue

            # Save image and mask
            # save_pth = '{key}_{int}.jpg'.format(_
            cv2.imwrite(os.path.join(image_output_path, f"{key}_{i}.jpg"), image)
            cv2.imwrite(os.path.join(mask_output_path, f"{key}_{i}.png"), mask)


In [ ]:
main(dataset_val,'../../train')
main(dataset_val,'../../val')
main(dataset_test,'../../test')